In [6]:
from datamodel import OrderDepth, UserId, TradingState, Order, Trade
from typing import List
import string

class Trader:
  

    
    def run(self, state: TradingState):
        print("traderData: " + state.traderData)
        print("Observations: " + str(state.observations))

				# Orders to be placed on exchange matching engine
        result = {}
        for product in state.order_depths:
            order_depth: OrderDepth = state.order_depths[product]
            orders: List[Order] = []
            acceptable_price = 10  # Participant should calculate this value
            print("Acceptable price : " + str(acceptable_price))
            print("Buy Order depth : " + str(len(order_depth.buy_orders)) + ", Sell order depth : " + str(len(order_depth.sell_orders)))
    
            if len(order_depth.sell_orders) != 0:
                best_ask, best_ask_amount = list(order_depth.sell_orders.items())[0]
                if int(best_ask) < acceptable_price:
                    print("BUY", str(-best_ask_amount) + "x", best_ask)
                    orders.append(Order(product, best_ask, -best_ask_amount))
    
            if len(order_depth.buy_orders) != 0:
                best_bid, best_bid_amount = list(order_depth.buy_orders.items())[0]
                if int(best_bid) > acceptable_price:
                    print("SELL", str(best_bid_amount) + "x", best_bid)
                    orders.append(Order(product, best_bid, -best_bid_amount))
            
            result[product] = orders
    
		    # String value holding Trader state data required. 
				# It will be delivered as TradingState.traderData on next execution.
        traderData = "SAMPLE" 
        
				# Sample conversion request. Check more details below. 
        conversions = 1
        return result, conversions, traderData

In [3]:
from typing import Dict, List
from datamodel import Order, TradingState, Trade, OrderDepth, UserId, Observation, ObservationValue, ConversionObservation

class TutorialTrader:
    def __init__(self):
        self.position_limits = {
            'AMETHYSTS': 20,
            'STARFRUIT': 20
        }
        self.price_history = {
            'STARFRUIT': []
        }

    def run(self, state: TradingState):
        result = {}
        conversions = 0
        traderData = ""

        # Update price history with trades
        self.update_price_history(state.market_trades.get('STARFRUIT', []))

        for product, order_depth in state.order_depths.items():
            orders = []

            if product == 'AMETHYSTS':
                continue  # Skip AMETHYSTS since it's constant
            
            # Calculate moving average of historical prices for STARFRUIT
            moving_avg = self.calculate_moving_average('STARFRUIT')

            # Set buy and sell prices based on moving average
            buy_price = round(moving_avg * 0.95)  # Buy at 5% below moving average
            sell_price = round(moving_avg * 1.05)  # Sell at 5% above moving average

            # Create buy order if position allows and buy price is lower than best ask
            if self.check_position_limit(state, product, 1) and buy_price < self.get_best_ask(order_depth):
                orders.append(Order(product, buy_price, 1))

            # Create sell order if position allows and sell price is higher than best bid
            if self.check_position_limit(state, product, -1) and sell_price > self.get_best_bid(order_depth):
                orders.append(Order(product, sell_price, -1))

            result[product] = orders

        return result, conversions, traderData

    def calculate_moving_average(self, product: str) -> float:
        """Calculates the moving average of historical prices for the given product."""
        history = self.price_history[product]
        if history:
            return sum(history) / len(history)
        else:
            return 0  # If no history, return 0

    def update_price_history(self, trades: List[Trade]):
        """Updates the price history based on the trades."""
        for trade in trades:
            if trade.symbol == 'STARFRUIT':
                self.price_history['STARFRUIT'].append(trade.price)

    def get_best_ask(self, order_depth: OrderDepth) -> int:
        """Returns the best ask price from the order depth."""
        if order_depth.sell_orders:
            return min(order_depth.sell_orders.keys())
        else:
            return float('inf')  # Return infinity if no asks

    def get_best_bid(self, order_depth: OrderDepth) -> int:
        """Returns the best bid price from the order depth."""
        if order_depth.buy_orders:
            return max(order_depth.buy_orders.keys())
        else:
            return float('-inf')  # Return negative infinity if no bids

    def check_position_limit(self, state: TradingState, product: str, additional_quantity: int) -> bool:
        """Checks if placing additional_quantity of orders respects position limits."""
        current_position = state.position.get(product, 0)
        position_limit = self.position_limits.get(product, 0)
        return abs(current_position + additional_quantity) <= position_limit

# Sample usage
if __name__ == "__main__":
    # Create an instance of TutorialTrader
    trader = TutorialTrader()

    # Sample usage with a TradingState object
    # Create a TradingState object with mock data
    starfruit_order_depth = OrderDepth()
    starfruit_order_depth.sell_orders = {20: -8, 21: -4}
    starfruit_order_depth.buy_orders = {18: 3, 19: 6}

    order_depths = {'STARFRUIT': starfruit_order_depth}
    position = {'AMETHYSTS': 10, 'STARFRUIT': -5}
    observations = Observation({}, {})
    market_trades = {'STARFRUIT': [Trade('STARFRUIT', 20, 8), Trade('STARFRUIT', 21, 4)]}
    trading_state = TradingState("Mock Trader Data", 1000, {}, order_depths, {}, market_trades, position, observations)

    # Run the trading algorithm
    result, conversions, traderData = trader.run(trading_state)

    # Print the resulting orders
    for product, orders in result.items():
        print(f"Orders for {product}: {orders}")


Orders for STARFRUIT: [(STARFRUIT, 19, 1), (STARFRUIT, 22, -1)]


In [4]:
from typing import Dict, List
from datamodel import OrderDepth, TradingState, Order


class Trader:
    def __init__(self):
        self.starfruit_prices = []

    def calculate_moving_average(self, period: int) -> float:
        if len(self.starfruit_prices) < period:
            return None
        else:
            return sum(self.starfruit_prices[-period:]) / period

    def run(self, state: TradingState) -> Dict[str, List[Order]]:
        result = {"AMETHYSTS": [], "STARFRUIT": []}

        # Extracting order depths for AMETHYSTS and STARFRUIT
        amethysts_orders = state.order_depths.get("AMETHYSTS", None)
        starfruit_orders = state.order_depths.get("STARFRUIT", None)

        if amethysts_orders and starfruit_orders:
            # Determine the current position in AMETHYSTS
            amethysts_position = state.position.get("AMETHYSTS", 0)

            # Determine the current position in STARFRUIT
            starfruit_position = state.position.get("STARFRUIT", 0)

            # Update starfruit_prices with the latest price
            latest_price = starfruit_orders.last_price
            self.starfruit_prices.append(latest_price)

            # Calculate the moving average over a period of 5 time steps
            moving_average = self.calculate_moving_average(5)

            if moving_average is not None:
                # Check if we can buy more STARFRUIT
                if starfruit_position < 20 and latest_price < moving_average:
                    # Look at the top buy order for STARFRUIT
                    best_bid_starfruit = max(starfruit_orders.buy_orders.keys())

                    # Check if the best bid is acceptable
                    if best_bid_starfruit < 10:
                        # Calculate how much STARFRUIT we can buy
                        max_buy_starfruit = min(
                            20 - starfruit_position,
                            starfruit_orders.buy_orders[best_bid_starfruit],
                        )

                        # Place the buy order for STARFRUIT
                        result["STARFRUIT"].append(
                            Order("STARFRUIT", best_bid_starfruit, max_buy_starfruit)
                        )

                # Check if we can sell some STARFRUIT
                if starfruit_position > -20 and latest_price > moving_average:
                    # Look at the top sell order for STARFRUIT
                    best_ask_starfruit = min(starfruit_orders.sell_orders.keys())

                    # Check if the best ask is acceptable
                    if best_ask_starfruit > 10:
                        # Calculate how much STARFRUIT we can sell
                        max_sell_starfruit = min(
                            starfruit_position + 20,
                            starfruit_orders.sell_orders[best_ask_starfruit],
                        )

                        # Place the sell order for STARFRUIT
                        result["STARFRUIT"].append(
                            Order("STARFRUIT", best_ask_starfruit, -max_sell_starfruit)
                        )

        return result


In [5]:
import os 
import pandas as pd

In [14]:
data = pd.DataFrame(pd.read_csv("./data.csv"))

In [1]:
from datamodel import OrderDepth, UserId, TradingState, Order
from typing import List
import numpy as np
import statistics


class Trader:
    def calculate_sma(self, prices, window):
        if len(prices) < window:
            return sum(prices) / len(prices)
        else:
            return sum(prices[-window:]) / window

    def mean(self, prices):
        return np.mean(prices)

    def run(self, state):
        # empty dictionary to later store the results and trades
        result = {}
        totalPnl = 0
        price_data_1 = []
        price_data_2 = []

        for product in state.order_depths:
            order_depth = state.order_depths[product]
            orders = []
            pnl = 0

            if product == "AMETHYSTS":
                am_prices_sell = min(list(order_depth.sell_orders.keys()))
                am_prices_buy = max(list(order_depth.buy_orders.keys()))
                a_price = (am_prices_buy + am_prices_sell) / 2
                price_data_1.append(a_price)

                mean = self.mean(price_data_1)

                if len(order_depth.sell_orders) != 0:
                    best_ask = min(
                        order_depth.sell_orders, key=order_depth.sell_orders.get
                    )
                    best_ask_amount = order_depth.sell_orders[best_ask]

                    # buy orders
                    if best_ask < mean and state.position[product] < 20:
                        state.postion[product] = (
                            state.postion[product] - best_ask_amount
                        )
                        orders.append(Order(product, best_ask, -best_ask_amount))
                        pnl += abs(best_ask * best_ask_amount)

                if len(order_depth.buy_orders) != 0:
                    best_bid = max(
                        order_depth.buy_orders, key=order_depth.buy_orders.get
                    )
                    best_bid_amount = order_depth.buy_orders[best_ask]

                    # the buying amount of the asset is more than sma
                    # means we can sell it at lower price
                    # so we place a sell order

                    if best_bid > mean and state.position[product] > 0:
                        state.postion[product] = (
                            state.postion[product] - best_bid_amount
                        )
                        orders.append(Order(product, best_bid, -best_bid_amount))
                        pnl += abs(best_bid * best_bid_amount)

            if product == "STARFRUIT":
                star_prices_sell = min(list(order_depth.sell_orders.keys()))
                star_prices_buy = max(list(order_depth.buy_orders.keys()))
                s_price = (star_prices_buy + star_prices_sell) / 2
                price_data_2.append(s_price)
                sma = self.calculate_sma(price_data_2, window=5)

                if len(order_depth.sell_orders) != 0:
                    best_ask = min(
                        order_depth.sell_orders, key=order_depth.sell_orders.get
                    )
                    best_ask_amount = order_depth.sell_orders[best_ask]

                    # the selling amount of the asset is less than sma
                    # means we can buy it at lower price
                    # so we place a buy order

                    if best_ask < sma and state.position["STARFRUIT"] < 20:
                        state.postion["STARFRUIT"] = (
                            state.postion["STARFRUIT"] - best_ask_amount
                        )
                        orders.append(Order("STARFRUIT", best_ask, -best_ask_amount))
                        pnl += abs(best_ask * best_ask_amount)

                # Generating sell signal
                if len(order_depth.buy_orders) != 0:
                    best_bid = max(
                        order_depth.buy_orders, key=order_depth.buy_orders.get
                    )
                    best_bid_amount = order_depth.buy_orders[best_ask]

                    # the buying amount of the asset is more than sma
                    # means we can sell it at lower price
                    # so we place a sell order

                    if best_bid > sma and state.position["STARFRUIT"] > 0:
                        state.postion["STARFRUIT"] = (
                            state.postion["STARFRUIT"] - best_bid_amount
                        )
                        orders.append(Order("STARFRUIT", best_bid, -best_bid_amount))
                        pnl += abs(best_bid * best_bid_amount)

            result[product] = orders
            totalPnl += pnl

        return result
